In [ ]:
from deepface import DeepFace
import cv2
import time

# IP Webcam URL
url = "http://192.168.100.9:8080/video"
cap = cv2.VideoCapture(url)


face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

if not cap.isOpened():
    print("Error: Could not open video stream")
    exit()

frame_count = 0
emotion = "Detecting..."
confidence = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame_count += 1

   
    frame = cv2.resize(frame, (640, 480))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]

        # Run DeepFace only every 15 frames 
        if frame_count % 15 == 0:
            try:
                result = DeepFace.analyze(
                    face_img,
                    actions=['emotion'],
                    enforce_detection=False
                )

                # DeepFace now returns list
                if isinstance(result, list):
                    result = result[0]

                emotion = result['dominant_emotion']
                confidence = max(result['emotion'].values())

            except Exception as e:
                emotion = "Error"
                confidence = 0

        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display emotion above face
        label = f"{emotion} {confidence:.0f}%"
        cv2.putText(
            frame,
            label,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 255),
            2
        )

    cv2.imshow("DeepFace Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Failed to grab frame
